In [1]:
import pandas as pd

from agage_archive import Paths
from agage_archive.io import combine_datasets, read_agage, read_ale_gage, output_dataset
from agage_archive.run import run_individual_instrument, run_combined_instruments

path = Paths()

In [24]:
run_individual_instrument("GAGE")

In [3]:
run_combined_instruments()

IndexError: index 0 is out of bounds for axis 0 with size 0